In [81]:
import cv2 as cv
import numpy as np
import csv
from scipy.ndimage.filters import gaussian_filter

In [82]:
#folderPath = 'C:/Users/jb83292/source/repos/edagmatse/Messepraesenter/data/Videos_Tabletennisball/'
folderPath =  'C:/Users/jb83292/source/repos/JoBo33/Kalman-Trajectory-Parabola/data/'
videoName = '60fps_not_flat'
#videoPath = folderPath + videoName + '.mp4'
videoPath = folderPath + videoName + '.MOV'
SaveTestdataPath = 'C:/Users/jb83292/source/repos/JoBo33/Kalman-Trajectory-Parabola/data/Test_CSV_Files/'



In [83]:
# mirror y coordinates at y / 2
def TransformYCoordinate(yo, frameSize):
    ymax = frameSize[1]
    ym = ymax / 2
    dy = ym - yo
    return dy + ym

In [84]:
REDU = 8

def rgbh(xs,mask):
	def normhist(x): return x / np.sum(x)
 

	def h(rgb):
		return cv.calcHist([rgb]
					, [0, 1, 2]
					, imCropMask
					, [256//REDU, 256//REDU, 256//REDU]
					, [0, 256] + [0, 256] + [0, 256]
					)
		
	return normhist(sum(map(h, xs)))
  
def smooth(s,x):
    return gaussian_filter(x,s,mode='constant')

bgsub = cv.createBackgroundSubtractorMOG2(500, 60, True) #Der Wert der Schwelle kann variieren(60)
cap = cv.VideoCapture(videoPath)


key = 0

kernel = np.ones((3,3),np.uint8)
crop = False
camshift = False

termination = (cv.TERM_CRITERIA_EPS | cv.TERM_CRITERIA_COUNT, 10, 1)

font = cv.FONT_HERSHEY_SIMPLEX

pause= False

P = np.diag([1000,1000,1000,1000])**2


frameSize = 1100, 650

listCenterX=[]
listCenterY=[]
listCenterTransformedY = []


while(True):

	key = cv.waitKey(1) & 0xFF
	if key== ord("c"): crop = True
	if key== ord("p"): P = np.diag([100,100,100,100])**2
	if key==27: break
	if key==ord(" "): pause =not pause
	if(pause): continue

	ret, frame = cap.read()
	#frame=cv.resize(frame,(800,600))
	if(np.shape(frame) == ()) : break
	frame=cv.resize(frame,(frameSize[0],frameSize[1]))
	
	bgs = bgsub.apply(frame)
	
	bgs = cv.erode(bgs,kernel,iterations = 1)
	bgs = cv.medianBlur(bgs,3)
	bgs = cv.dilate(bgs,kernel,iterations=2)
	
	bgs = (bgs > 200).astype(np.uint8)*255
	colorMask = cv.bitwise_and(frame,frame,mask = bgs)

	if(crop):
		fromCenter= False
		img = colorMask
		r = cv.selectROI(img, fromCenter)
		imCrop = img[int(r[1]):int(r[1]+r[3]), int(r[0]):int(r[0]+r[2])]
		crop = False
		camshift = True
		imCropMask = cv.cvtColor(imCrop, cv.COLOR_BGR2GRAY)
		ret,imCropMask = cv.threshold(imCropMask,30,255,cv.THRESH_BINARY)
		his = smooth(1,rgbh([imCrop],imCropMask))
		roiBox = (int(r[0]), int(r[1]),int(r[2]), int(r[3]))
		cv.destroyWindow("ROI selector")

		
	if(camshift):
		cv.putText(frame,'Center roiBox',(0,10), font, 0.5,(0,255,0),2,cv.LINE_AA)
		cv.putText(frame,'Estimated position',(0,30), font,0.5,(255,255,0),2,cv.LINE_AA)
		cv.putText(frame,'Prediction',(0,50), font, 0.5,(0,0,255),2,cv.LINE_AA)

		rgbr = np.floor_divide( colorMask , REDU)
		r,g,b = rgbr.transpose(2,0,1)
		l = his[r,g,b]
		maxl = l.max()
		aa=np.clip((1*l/maxl*255),0,255).astype(np.uint8)
		
		(rb, roiBox) = cv.CamShift(l, roiBox, termination)
		cv.ellipse(frame, rb, (0, 255, 0), 2)

		xo=int(roiBox[0]+roiBox[2]/2)
		yo=int(roiBox[1]+roiBox[3]/2)
		error=(roiBox[3])
		
		if(yo<error or bgs.sum()<50):
			mm = False
		else:
			mm = True
			
		if(mm):
			listCenterX.append(xo)
			listCenterY.append(yo)
			listCenterTransformedY.append(TransformYCoordinate(yo, frameSize))
		
		for n in range(len(listCenterX)): 
			cv.circle(frame,(int(listCenterX[n]),int(listCenterY[n])),3,(0, 255, 0),-1)
			

		if (len(listCenterY)>4):
			if ((listCenterY[-5] < listCenterY[-4]) and(listCenterY[-4] <listCenterY[-3]) and (listCenterY[-3] > listCenterY[-2]) and (listCenterY[-2] > listCenterY[-1])):
				print("REBOTE")
				break
	
	cv.imshow('ColorMask',colorMask)
	cv.imshow('mask', bgs)
	cv.imshow('Frame', frame)



C:\Users\jb83292\AppData\Local\Temp/ipykernel_28624/1047243255.py:92: RuntimeWarning: invalid value encountered in true_divide
  aa=np.clip((1*l/maxl*255),0,255).astype(np.uint8)


In [85]:
# Create array of points 
xy= [[0 for x in range(2)] for y in range(len(listCenterX))] 
for i in range(len(listCenterX)):
    xy[i][0] = listCenterX[i]
    xy[i][1] = listCenterY[i]



In [86]:
# Create array of points with transformed y coordinates
xyT= [[0 for x in range(2)] for y in range(len(listCenterX))] 
for i in range(len(listCenterX)):
    xyT[i][0] = listCenterX[i]
    xyT[i][1] = listCenterTransformedY[i]

In [87]:
# open the file in the write mode
with open(SaveTestdataPath + videoName + '.csv', 'w') as f:

    # create the csv writer
    writer = csv.writer(f, delimiter= ';')
    for i in range(len(listCenterX)):
        # write a row to the csv file     
        writer.writerow( xy[i] )



In [88]:
# open the file in the write mode
with open(SaveTestdataPath + videoName + 'Transformed.csv', 'w') as f:

    # create the csv writer
    writer = csv.writer(f, delimiter= ';')
    for i in range(len(listCenterX)):
        # write a row to the csv file     
        writer.writerow( xyT[i] )